In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys
import random

In [2]:
df = pd.read_csv("dataset/resize_borvo.csv") 

grid = {}
for _, row in df.iterrows():
    grid[(int(row["i"]), int(row["j"]))] = {"H": row["H"], "W": row["W"]}

In [3]:
# Fungsi ambil tetangga 8 arah (termasuk diagonal)
def get_all_neighbors(pos):
    i, j = pos
    directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1), (1, 0), (1, 1)
    ]
    neighbors = [(i + di, j + dj) for di, dj in directions]
    return [n for n in neighbors if n in grid]

# Fungsi hitung skor W/E
def get_score(current_pos, neighbor_pos):
    H1 = grid[current_pos]["H"]
    H2 = grid[neighbor_pos]["H"]
    S = H1 - H2

    if S > 0.5:
        return -np.inf  # Tidak aman → skip

    E = 1 + 1 * abs(S)
    W = grid[neighbor_pos]["W"]
    return W / E

In [4]:
def get_state_quality(node):
    kualitas = grid[node]["H"] - (0.5 * grid[node]["W"])
    return kualitas
def simulated_annealing_standard(start, Tmax=1000, Tmin=1.0, alpha=0.95, max_iter=1000):
    """
    Versi SA yang lebih standar, sekarang juga mengembalikan konsumsi energi.
    """
    if start not in grid:
        print(f"Titik {start} tidak ditemukan dalam dataset.")
        return [], [], [] # Kembalikan 3 list kosong

    current_pos = start
    path = [current_pos]
    
    # Siapkan list untuk performa (kualitas state) dan konsumsi energi
    performance_data = [get_state_quality(current_pos)]
    energy_consumption = [] # <-- LIST BARU UNTUK ENERGI

    current_quality = get_state_quality(current_pos)
    T = Tmax

    for _ in range(max_iter):
        if T < Tmin:
            break

        valid_neighbors = [n for n in get_all_neighbors(current_pos) if n not in path]
        if not valid_neighbors:
            break
        
        next_pos = random.choice(valid_neighbors)
        
        safety_check_score = get_score(current_pos, next_pos)
        if safety_check_score == -np.inf:
            continue

        next_quality = get_state_quality(next_pos)
        delta_quality = next_quality - current_quality

        # Kriteria penerimaan Metropolis
        move_accepted = False
        if delta_quality > 0:
            move_accepted = True
        elif random.random() < np.exp(delta_quality / T):
            move_accepted = True
        
        if move_accepted:
            # === TAMBAHKAN PERHITUNGAN ENERGI DI SINI ===
            H1 = grid[current_pos]["H"]
            H2 = grid[next_pos]["H"]
            energy_for_this_step = 1 + abs(H1 - H2)
            energy_consumption.append(energy_for_this_step)
            # ============================================

            # Update state
            current_pos = next_pos
            current_quality = next_quality
            path.append(current_pos)
            performance_data.append(current_quality)
        
        T *= alpha
    
    # KEMBALIKAN 3 NILAI SEKARANG
    return path, performance_data, energy_consumption

In [5]:
# Titik uji coba
test_points = [
    (540/2, 648/2),
    (20/2, 502/2),
    (410//2, 915//2),
    (529, 357)
]

# Jalankan algoritma untuk semua titik uji
for idx, point in enumerate(test_points, 1):
    print(f"\n=== Titik {idx} - Start di {point} ===")
    path = simulated_annealing_standard(point)
    print(f"Jumlah langkah: {len(path)}")
    print("Jalur:")
    for p in path:
        print(p)


=== Titik 1 - Start di (270.0, 324.0) ===
Jumlah langkah: 3
Jalur:
[(270.0, 324.0), (269.0, 324.0), (269.0, 323.0), (270.0, 323.0), (269.0, 322.0), (268.0, 321.0), (268.0, 320.0), (267.0, 319.0), (267.0, 318.0), (268.0, 318.0), (269.0, 318.0), (268.0, 317.0), (269.0, 317.0), (269.0, 316.0), (270.0, 316.0), (271.0, 317.0), (272.0, 317.0), (273.0, 316.0), (273.0, 315.0), (274.0, 316.0), (274.0, 315.0), (274.0, 314.0), (274.0, 313.0), (275.0, 314.0), (276.0, 313.0), (277.0, 314.0), (278.0, 315.0), (277.0, 315.0), (276.0, 316.0), (275.0, 316.0), (275.0, 315.0), (276.0, 315.0), (277.0, 316.0), (278.0, 316.0), (277.0, 317.0), (278.0, 317.0), (279.0, 316.0), (279.0, 317.0), (279.0, 318.0), (280.0, 317.0), (281.0, 318.0), (282.0, 319.0), (283.0, 319.0), (282.0, 318.0), (283.0, 318.0), (282.0, 317.0), (283.0, 316.0), (282.0, 316.0), (281.0, 316.0), (282.0, 315.0), (281.0, 314.0), (280.0, 313.0), (279.0, 314.0), (279.0, 313.0), (278.0, 313.0), (277.0, 312.0), (278.0, 311.0), (278.0, 310.0), (27

In [6]:
# =====================================================================
# --- BAGIAN 1: KONFIGURASI SIMULASI ---
# =====================================================================
# Daftar titik awal untuk disimulasikan
test_points = [
    (270, 324),
    (10, 251),
    (410//2, 915//2)
]
nama_file_peta = 'europa_map/raw/resize_borvo_mensa.jpg' # Pastikan nama peta sudah benar

# Loop untuk menjalankan simulasi untuk setiap titik awal
for idx, titik_awal in enumerate(test_points, 1):
    nama_file_output = f'simulasi_SA_{idx}_start_at_{titik_awal}.gif'

    print("\n" + "="*60)
    print(f"--- MEMULAI SIMULASI SA {idx} DARI {len(test_points)} | START: {titik_awal} ---")
    print("="*60)

    # =====================================================================
    # --- BAGIAN 2: PERSIAPAN DATA ---
    # =====================================================================
    print("1. Menjalankan algoritma Simulated Annealing...")
    try:
        # PERHATIKAN: Sekarang kita menerima 3 variabel
        jalur, data_performa, energi_per_langkah = simulated_annealing_standard(titik_awal)
    except Exception as e:
        print(f"!!! TERJADI ERROR SAAT MENJALANKAN ALGORITMA: {e}")
        continue # Lanjut ke titik berikutnya jika error

    if not jalur or len(jalur) < 2:
        print(f"!!! ALGORITMA HANYA MENGHASILKAN {len(jalur)} TITIK. Melanjutkan ke titik berikutnya.")
        continue

    # HITUNG TOTAL ENERGI
    total_energi = sum(energi_per_langkah)

    print(f"2. Algoritma selesai. Ditemukan jalur dengan {len(jalur)} langkah.")
    print(f"   >> Total Konsumsi Energi: {total_energi:.2f}") # Tampilkan di terminal
    langkah_total = range(len(jalur))

    # =====================================================================
    # --- BAGIAN 3: PERSIAPAN PLOT ---
    # =====================================================================
    print("3. Menyiapkan 'panggung' untuk rendering animasi...")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    fig.suptitle(f'Simulasi Robot SA ke {idx} di Europa (Start: {titik_awal})', fontsize=16)

    try:
        img = plt.imread(nama_file_peta)
        ax1.imshow(img)
    except FileNotFoundError:
        ax1.set_facecolor('black')

    jejak_robot, = ax1.plot([], [], 'm-', lw=2, alpha=0.8, label='Jejak SA', zorder=5)
    posisi_robot, = ax1.plot([], [], 'yo', ms=8, label='Posisi Robot', zorder=10) 
    ax1.legend()

    ax2.set_title('Grafik Performa SA')
    ax2.set_xlabel('Langkah')
    ax2.set_ylabel('Kualitas State (H - 0.5*W)')
    ax2.set_xlim(0, len(jalur))
    if data_performa:
        ax2.set_ylim(min(data_performa) - 5, max(data_performa) + 5)
    ax2.grid(True)
    grafik_performa, = ax2.plot([], [], 'c-', marker='o', ms=4)
    
    # TAMBAHKAN TEKS ENERGI PADA GRAFIK
    ax2.text(0.95, 0.95, f'Total Energi: {total_energi:.2f}',
             transform=ax2.transAxes, fontsize=12,
             verticalalignment='top', horizontalalignment='right',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
             
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # =====================================================================
    # --- BAGIAN 4: FUNGSI UPDATE (TIDAK BERUBAH) ---
    # =====================================================================
    def update(frame):
        if frame % 20 == 0:
            print(f"   ...merender frame {frame}/{len(jalur)}")
        
        path_sejauh_ini = jalur[:frame+1]
        koordinat_j = [titik[1] for titik in path_sejauh_ini]
        koordinat_i = [titik[0] for titik in path_sejauh_ini]
        
        jejak_robot.set_data(koordinat_j, koordinat_i)
        posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])
        grafik_performa.set_data(langkah_total[:frame+1], data_performa[:frame+1])
        ax1.set_title(f'Peta Penjelajahan (Langkah: {frame})')
        return jejak_robot, posisi_robot, grafik_performa,

    # =====================================================================
    # --- BAGIAN 5: RENDER DAN SIMPAN KE FILE GIF ---
    # =====================================================================
    print("4. Memulai proses rendering animasi ke file GIF. Ini mungkin butuh waktu...")
    ani = animation.FuncAnimation(fig, update, frames=len(jalur), blit=False)

    try:
        ani.save(nama_file_output, writer='pillow', fps=20)
        
        print("\n" + "#"*60)
        print(f"###   BERHASIL! Animasi telah disimpan sebagai '{nama_file_output}'   ###")
        print("#"*60)

    except Exception as e:
        print(f"\n!!! GAGAL MENYIMPAN ANIMASI: {e}")
    
    plt.close(fig)

print("\nSEMUA SIMULASI SELESAI.")


--- MEMULAI SIMULASI SA 1 DARI 3 | START: (270, 324) ---
1. Menjalankan algoritma Simulated Annealing...
2. Algoritma selesai. Ditemukan jalur dengan 125 langkah.
   >> Total Konsumsi Energi: 156.12
3. Menyiapkan 'panggung' untuk rendering animasi...
4. Memulai proses rendering animasi ke file GIF. Ini mungkin butuh waktu...
   ...merender frame 0/125
   ...merender frame 0/125


C:\Users\Rhama\AppData\Local\Temp\ipykernel_9396\2348963335.py:88: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])


   ...merender frame 20/125
   ...merender frame 40/125
   ...merender frame 60/125
   ...merender frame 80/125
   ...merender frame 100/125
   ...merender frame 120/125

############################################################
###   BERHASIL! Animasi telah disimpan sebagai 'simulasi_SA_1_start_at_(270, 324).gif'   ###
############################################################

--- MEMULAI SIMULASI SA 2 DARI 3 | START: (10, 251) ---
1. Menjalankan algoritma Simulated Annealing...
2. Algoritma selesai. Ditemukan jalur dengan 45 langkah.
   >> Total Konsumsi Energi: 61.07
3. Menyiapkan 'panggung' untuk rendering animasi...
4. Memulai proses rendering animasi ke file GIF. Ini mungkin butuh waktu...
   ...merender frame 0/45
   ...merender frame 0/45


C:\Users\Rhama\AppData\Local\Temp\ipykernel_9396\2348963335.py:88: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])


   ...merender frame 20/45
   ...merender frame 40/45

############################################################
###   BERHASIL! Animasi telah disimpan sebagai 'simulasi_SA_2_start_at_(10, 251).gif'   ###
############################################################

--- MEMULAI SIMULASI SA 3 DARI 3 | START: (205, 457) ---
1. Menjalankan algoritma Simulated Annealing...
2. Algoritma selesai. Ditemukan jalur dengan 64 langkah.
   >> Total Konsumsi Energi: 80.15
3. Menyiapkan 'panggung' untuk rendering animasi...
4. Memulai proses rendering animasi ke file GIF. Ini mungkin butuh waktu...
   ...merender frame 0/64
   ...merender frame 0/64


C:\Users\Rhama\AppData\Local\Temp\ipykernel_9396\2348963335.py:88: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])


   ...merender frame 20/64
   ...merender frame 40/64
   ...merender frame 60/64

############################################################
###   BERHASIL! Animasi telah disimpan sebagai 'simulasi_SA_3_start_at_(205, 457).gif'   ###
############################################################

SEMUA SIMULASI SELESAI.
